In [1]:
from collections import defaultdict
from datetime import datetime

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import signal
import matplotlib
import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
DISTRIB_DELTA = 1
DISTRIB_MIN = -200
DISTRIB_MAX = -DISTRIB_MIN + DISTRIB_DELTA
DISTRIB_GRID = np.arange(DISTRIB_MIN, DISTRIB_MAX, DISTRIB_DELTA)

class Distrib:
    def __init__(self, pmf):
        self.pmf = pmf
        
    def __neg__(self):
        return Distrib(self.pmf[::-1])
        
    def __add__(self, other):
        conv_pmf = signal.fftconvolve(self.pmf, other.pmf, 'same')
        return Distrib(conv_pmf)
    
    def __sub__(self, other):
        return self + (-other)
    
    def fig(self):
        fig, ax = plt.subplots()
        cond = self.pmf / DISTRIB_DELTA > 0.001
        ax.plot(DISTRIB_GRID[cond], self.pmf[cond] / DISTRIB_DELTA)
        fig.show()
        
    @staticmethod
    def fig_list(distribs):
        fig, ax = plt.subplots()
        cond = DISTRIB_GRID > DISTRIB_MAX
        for d in distribs:
            cond = np.logical_or(cond, d.pmf / DISTRIB_DELTA > 0.001)
        for d in distribs:
            ax.plot(DISTRIB_GRID[cond], d.pmf[cond] / DISTRIB_DELTA)
        fig.show()
        
    def win_prob(self):
        return sum(self.pmf[DISTRIB_GRID >= 0]) / sum(self.pmf)
    
    def mean(self):
        return np.average(DISTRIB_GRID, weights=self.pmf)
    
    def sd(self):
        return np.average((DISTRIB_GRID - self.mean()) ** 2, weights=self.pmf) ** 0.5
    
    def __repr__(self):
        return "Distrib (" + str(round(self.mean(), 2)) + "," + str(round(self.sd(), 2)) + ")"
        
        
def t_distrib(df, loc, scale):
    pmf = stats.t.pdf(DISTRIB_GRID, df=df, loc=loc, scale=scale) * DISTRIB_DELTA
    return Distrib(pmf)


def n_distrib(loc, scale):
    pmf = stats.norm.pdf(DISTRIB_GRID, loc=loc, scale=scale) * DISTRIB_DELTA
    return Distrib(pmf)


class Rating:
    def __init__(self, prior_n=0, prior_mean=0, prior_sd=0, decay=0.9):
        self.mu = prior_mean
        self.v = prior_n
        self.alpha = prior_n / 2
        self.beta = prior_sd * prior_sd * prior_n * prior_n / (prior_n + 1) / 2
        
        self.d = decay
        
        self._history = []
        
        self.record()
        
    def decay(self):
        self.v *= self.d
        self.alpha *= self.d
        self.beta *= self.d
        
    def record(self, x=0):
        mean, sd = self.mean(), self.sd()
        self._history.append({
            "match": len(self._history),
            "reading": x,
            "lower": mean - 2 * sd,
            "mean": mean,
            "upper": mean + 2 * sd,
        })
        
        if len(self._history) > 1:
            self._history[-2]["reading"] = self._history[-1]["reading"]
        
    def history(self, key):
        return [x[key] for x in self._history]
    
    def plot_history(self, fig=None, ax=None, i=0):
        colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
        if ax is None:
            fig, ax = plt.subplots(figsize=(9, 6))
        ax.scatter(self.history("match"), self.history("reading"), label="Readings")
        ax.plot(self.history("match"), self.history("mean"), label="Bayesian")
        ax.fill_between(
            self.history("match"), 
            self.history("lower"), 
            self.history("upper"), 
            color=colors[i], 
            alpha=0.2, 
            label="CI"
        )
        ax.legend()
        if fig is not None:
            fig.show()
        
    def add(self, x):
        self.decay()
        self.mu = (self.v * self.mu + x) / (self.v + 1)
        self.v += 1
        self.alpha += 0.5
        self.beta += self.v / (self.v + 1) * (x - self.mu) ** 2 / 2
        self.record(x)
        
    def df(self):
        return self.v
        
    def mean(self):
        return self.mu
    
    def sd(self):
        if self.v == 0:
            return 0
        return (self.beta * (self.v + 1) / (self.alpha * self.v)) ** 0.5
    
    def n_distrib(self):
        return n_distrib(self.mean(), self.sd())
    
    def t_distrib(self):
        return self.n_distrib()
        # return t_distrib(self.df(), self.mean(), self.sd())
    
    def __repr__(self):
        return "Rating (" + str(round(self.mean(), 2)) + ", " + str(round(self.sd(), 2)) + ")"

In [3]:
years_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/years.csv")

In [4]:
team_years_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/team_years.csv")

In [5]:
events_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/events.csv")

In [6]:
matches_df = pd.read_csv("https://raw.githubusercontent.com/avgupta456/statbotics-csvs/main/matches.csv")

In [7]:
events_df[events_df.key == "2022gal"]

,key,year,name,time,state,country,district,type,week,status,...,elo_acc,elo_mse,opr_acc,opr_mse,mix_acc,mix_mse,rp1_acc,rp1_mse,rp2_acc,rp2_mse
1536,2022gal,2022,Galileo Division,1650427200,TX,USA,NaN,3,8,Completed,...,0.7483,0.1517,0.7552,0.1673,0.7343,0.1544,0.8504,0.1101,0.7677,0.1797


In [8]:
start_oprs = defaultdict(lambda: 13.8)
start_elos = defaultdict(lambda: 1500)
for _, t in team_years_df[(team_years_df.year == 2022)].iterrows():
    start_oprs[t.team] = t.opr_start
    start_elos[t.team] = t.elo_start

In [9]:
event_matches_df = matches_df[(matches_df.year == 2022) & (matches_df.comp_level == "qm")].sort_values(by=["time"])

display(event_matches_df)

,key,year,event,comp_level,set_number,match_number,status,red,red_elo_sum,red_opr_sum,...,blue_teleop_1,blue_teleop_2,blue_1,blue_2,blue_teleop,blue_endgame,blue_no_fouls,blue_fouls,blue_rp_1,blue_rp_2
154345,2022nhgrs_qm1,2022,2022nhgrs,qm,1,1,Completed,"7913,131,467",4617,29.69,...,2.0,0.0,4.0,0.0,2.0,0.0,6.0,0.0,0.0,0.0
151040,2022miket_qm1,2022,2022miket,qm,1,1,Completed,"6067,1506,6091",4705,42.20,...,11.0,0.0,13.0,0.0,11.0,6.0,23.0,0.0,0.0,0.0
150436,2022midet_qm1,2022,2022midet,qm,1,1,Completed,"5090,2224,3414",4591,43.95,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
147382,2022flwp_qm1,2022,2022flwp,qm,1,1,Completed,"59,744,1523",4853,66.27,...,1.0,0.0,5.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0
147393,2022flwp_qm2,2022,2022flwp,qm,1,2,Completed,"5410,7652,3627",4427,44.91,...,0.0,0.0,6.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145947,2022carv_qm125,2022,2022carv,qm,1,125,Completed,"1700,870,3654",4952,85.25,...,35.0,0.0,55.0,0.0,35.0,0.0,61.0,0.0,1.0,0.0
157580,2022tur_qm123,2022,2022tur,qm,1,123,Completed,"5484,5985,2539",4947,81.38,...,41.0,0.0,53.0,0.0,41.0,10.0,69.0,4.0,1.0,0.0
157581,2022tur_qm124,2022,2022tur,qm,1,124,Completed,"3792,3061,1391",5122,116.21,...,82.0,0.0,114.0,0.0,82.0,34.0,154.0,4.0,1.0,1.0
157582,2022tur_qm125,2022,2022tur,qm,1,125,Completed,"8006,4003,649",4983,99.21,...,42.0,0.0,58.0,0.0,42.0,20.0,84.0,0.0,1.0,1.0


In [10]:
elos = defaultdict(lambda: 1500)
for t in start_elos:
    elos[t] = start_elos[t]

mean = 13.8
decay = 0.7
ratings = defaultdict(lambda: Rating(prior_n=3, prior_mean=mean, prior_sd=mean, decay=decay))
auto_ratings = defaultdict(lambda: Rating(prior_n=3, prior_mean=mean / 3, prior_sd = mean / 3 ** 0.5, decay=decay))
teleop_ratings = defaultdict(lambda: Rating(prior_n=3, prior_mean=mean / 3, prior_sd = mean / 3 ** 0.5, decay=decay))
endgame_ratings = defaultdict(lambda: Rating(prior_n=3, prior_mean=mean / 3, prior_sd = mean / 3 ** 0.5, decay=decay))
for t in start_oprs:
    ratings[t] = Rating(prior_n=3, prior_mean=start_oprs[t], prior_sd=mean, decay=decay)
    auto_ratings[t] = Rating(prior_n=3, prior_mean=start_oprs[t] / 3, prior_sd=mean / 3 ** 0.5, decay=decay)
    teleop_ratings[t] = Rating(prior_n=3, prior_mean=start_oprs[t] / 3, prior_sd=mean / 3 ** 0.5, decay=decay)
    endgame_ratings[t] = Rating(prior_n=3, prior_mean=start_oprs[t] / 3, prior_sd=mean / 3 ** 0.5, decay=decay)

count, acc, mse = 0, 0, 0
elo_count, elo_acc, elo_mse = 0, 0, 0
for _, match in event_matches_df.iterrows():
    red_teams = [int(x) for x in match.red.split(",")]
    blue_teams = [int(x) for x in match.blue.split(",")]
    
    red_means = [ratings[r].mean() for r in red_teams]
    blue_means = [ratings[b].mean() for b in blue_teams]
    red_sum = sum(red_means)
    blue_sum = sum(blue_means)
    
    red_distrib = ratings[red_teams[0]].t_distrib()
    for r in red_teams[1:]:
        red_distrib += ratings[r].t_distrib()
        
    blue_distrib = ratings[blue_teams[0]].t_distrib()
    for b in blue_teams[1:]:
        blue_distrib += ratings[b].t_distrib()
        
    red_auto_distrib = auto_ratings[red_teams[0]].t_distrib() + auto_ratings[red_teams[1]].t_distrib() + auto_ratings[red_teams[2]].t_distrib()
    red_teleop_distrib = teleop_ratings[red_teams[0]].t_distrib() + teleop_ratings[red_teams[1]].t_distrib() + teleop_ratings[red_teams[2]].t_distrib()
    red_endgame_distrib = endgame_ratings[red_teams[0]].t_distrib() + endgame_ratings[red_teams[1]].t_distrib() + endgame_ratings[red_teams[2]].t_distrib()
    red_total_distrib = red_auto_distrib + red_teleop_distrib + red_endgame_distrib
    
    blue_auto_distrib = auto_ratings[blue_teams[0]].t_distrib() + auto_ratings[blue_teams[1]].t_distrib() + auto_ratings[blue_teams[2]].t_distrib()
    blue_teleop_distrib = teleop_ratings[blue_teams[0]].t_distrib() + teleop_ratings[blue_teams[1]].t_distrib() + teleop_ratings[blue_teams[2]].t_distrib()
    blue_endgame_distrib = endgame_ratings[blue_teams[0]].t_distrib() + endgame_ratings[blue_teams[1]].t_distrib() + endgame_ratings[blue_teams[2]].t_distrib()
    blue_total_distrib = blue_auto_distrib + blue_teleop_distrib + blue_endgame_distrib
    
    
    print(red_teams, blue_teams)
    print("Red")
    print(match.red_auto, match.red_teleop, match.red_endgame)
    print(list(auto_ratings[t] for t in red_teams))
    print(list(teleop_ratings[t] for t in red_teams))
    print(list(endgame_ratings[t] for t in red_teams))
    print(sum(auto_ratings[t].mean() + teleop_ratings[t].mean() + endgame_ratings[t].mean() for t in red_teams))
    print(red_total_distrib)
    print(red_distrib)
    print("Blue")
    print(match.blue_auto, match.blue_teleop, match.blue_endgame)
    print(list(auto_ratings[t] for t in blue_teams))
    print(list(teleop_ratings[t] for t in blue_teams))
    print(list(endgame_ratings[t] for t in blue_teams))
    print(sum(auto_ratings[t].mean() + teleop_ratings[t].mean() + endgame_ratings[t].mean() for t in blue_teams))
    print(blue_total_distrib)
    print(blue_distrib)
    print(win_prob)
    print()
    
    red_error = match.red_no_fouls / red_sum
    for r, mean in zip(red_teams, red_means):
        ratings[r].add(mean * red_error)
    blue_error = match.blue_no_fouls / blue_sum
    for b, mean in zip(blue_teams, blue_means):
        ratings[b].add(mean * blue_error)
    
    for match_result, teams, ratings_dict in [
        (match.red_auto, red_teams, auto_ratings),
        (match.blue_auto, blue_teams, auto_ratings),
        (match.red_teleop, red_teams, teleop_ratings),
        (match.blue_teleop, blue_teams, teleop_ratings),
        (match.red_endgame, red_teams, endgame_ratings),
        (match.blue_endgame, blue_teams, endgame_ratings),
    ]:
        pred_results = [ratings_dict[t].mean() for t in teams]
        for t, mean in zip(teams, pred_results):
            method1 = mean * match_result / sum(pred_results)
            method2 = mean + (match_result - sum(pred_results)) / 3
            measurement = 0.5 * method1 + 0.5 * method2
            ratings_dict[t].add(measurement)
    
    win_prob = (red_distrib - blue_distrib).win_prob()
    count += 1
    acc += (win_prob > 0.5 and match.red_score > match.blue_score) or (win_prob < 0.5 and match.red_score < match.blue_score)
    mse += (1 - win_prob) ** 2 if match.red_score > match.blue_score else win_prob ** 2
    
    red_elo_sum = sum(elos[r] for r in red_teams)
    blue_elo_sum = sum(elos[b] for b in blue_teams)
    elo_win_prob = 1 / (10 ** ((blue_elo_sum - red_elo_sum) / 400) + 1)
    elo_count += 1
    elo_acc += (elo_win_prob > 0.5 and match.red_score > match.blue_score) or (elo_win_prob < 0.5 and match.red_score < match.blue_score)
    elo_mse += (1 - elo_win_prob) ** 2 if match.red_score > match.blue_score else elo_win_prob ** 2
    
    pred_win_margin = 4 / 1000 * (red_elo_sum - blue_elo_sum)
    win_margin = (match.red_no_fouls - match.blue_no_fouls) / 22
    update = 12 * (win_margin - pred_win_margin)
    for r in red_teams:
        elos[r] += update
    for b in blue_teams:
        elos[b] -= update    
    
print(count, acc / count, mse / count)
print(elo_count, elo_acc / elo_count, elo_mse / elo_count)

[7913, 131, 467] [5902, 151, 1922]
Red
10.0 24.0 15.0
[Rating (3.76, 7.97), Rating (4.54, 7.97), Rating (1.6, 7.97)]
[Rating (3.76, 7.97), Rating (4.54, 7.97), Rating (1.6, 7.97)]
[Rating (3.76, 7.97), Rating (4.54, 7.97), Rating (1.6, 7.97)]
29.689999999999998
Distrib (29.69,23.9)
Distrib (29.69,23.9)
Blue
4.0 2.0 0.0
[Rating (2.65, 7.97), Rating (4.81, 7.97), Rating (3.59, 7.97)]
[Rating (2.65, 7.97), Rating (4.81, 7.97), Rating (3.59, 7.97)]
[Rating (2.65, 7.97), Rating (4.81, 7.97), Rating (3.59, 7.97)]
33.129999999999995
Distrib (33.13,23.9)
Distrib (33.13,23.9)


NameError: name 'win_prob' is not defined

In [ ]:
elo_count, elo_acc, elo_mse = 0, 0, 0
for _, match in event_matches_df.iterrows():
    elo_win_prob = match.opr_win_prob
    elo_count += 1
    elo_acc += (elo_win_prob > 0.5 and match.red_score > match.blue_score) or (elo_win_prob < 0.5 and match.red_score < match.blue_score)
    elo_mse += (1 - elo_win_prob) ** 2 if match.red_score > match.blue_score else elo_win_prob ** 2
    
print(elo_count, elo_acc / elo_count, elo_mse / elo_count)

In [ ]:
ratings[1678].plot_history()
auto_ratings[1678].plot_history()

In [ ]:
fig, ax = plt.subplots()
ratings[254].plot_history(ax=ax, i=0)
ratings[1323].plot_history(ax=ax, i=1)
fig.show()

In [ ]:
print(ratings[254])
print(elos[254])

In [ ]:
print(ratings[1690])
print(elos[1690])

In [ ]:
print(ratings[1678])
print(elos[1678])

In [ ]:
t1 = ratings[1678].t_distrib()
t2 = ratings[254].t_distrib()
Distrib.fig_list([t1, t2])
print((t1 - t2).win_prob())

In [ ]:
data = sorted(elos.items(), key=lambda x: -x[1])[:10]
for x in data:
    print(x)
    
print()
    
data = sorted(endgame_ratings.items(), key=lambda x: -(x[1].mean() - x[1].sd()))[:20]
# data = sorted(ratings.items(), key=lambda x: -max(x[1].history("mean")))[:20]
for x in data:
    # print(x, max(x[1].history("mean")))
    print(x, x[1].mean() - x[1].sd())

In [ ]:
team = 67

print(ratings[team])
print(auto_ratings[team])
print(teleop_ratings[team])
print(endgame_ratings[team])
print(auto_ratings[team].mean() + teleop_ratings[team].mean() + endgame_ratings[team].mean())